In [68]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import torchvision.models as models

# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Freeze all parameters
for param in vgg16.parameters():
    param.requires_grad = False

# Modify last fully connected layer to match number of classes
num_classes = 25  # Change this to the number of classes in your dataset
vgg16.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes)

# Define transformations for the dataset
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
batch_size = 32
train_dataset = datasets.ImageFolder('/kaggle/input/cv-assignment/group_3/train', train_transform)
val_dataset = datasets.ImageFolder('/kaggle/input/cv-assignment/group_3/valid', val_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier[-1].parameters(), lr=0.001,weight_decay=0.0001)

# Train the model
num_epochs = 25
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)
for epoch in range(num_epochs):
    # Train for one epoch
    vgg16.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = vgg16(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_loader.dataset)

    # Validate after one epoch
    vgg16.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = vgg16(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
        val_loss /= len(val_loader.dataset)
        val_acc = correct.double() / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    checkpoint_path = f'/kaggle/working/epoch{epoch+1}.pt'
    torch.save(vgg16.state_dict(), checkpoint_path)

# Save the trained model
# torch.save(vgg16.state_dict(), '/kaggle/working/best.pth')



Epoch 1/25, Train Loss: 0.6829, Val Loss: 0.1448, Val Acc: 0.9760
Epoch 2/25, Train Loss: 0.2333, Val Loss: 0.0886, Val Acc: 0.9840
Epoch 3/25, Train Loss: 0.1803, Val Loss: 0.0672, Val Acc: 0.9840
Epoch 4/25, Train Loss: 0.1569, Val Loss: 0.0667, Val Acc: 0.9920
Epoch 5/25, Train Loss: 0.1532, Val Loss: 0.0747, Val Acc: 0.9840
Epoch 6/25, Train Loss: 0.1137, Val Loss: 0.0768, Val Acc: 0.9840
Epoch 7/25, Train Loss: 0.1133, Val Loss: 0.0558, Val Acc: 0.9840
Epoch 8/25, Train Loss: 0.1115, Val Loss: 0.0554, Val Acc: 0.9840
Epoch 9/25, Train Loss: 0.0896, Val Loss: 0.0433, Val Acc: 0.9920
Epoch 10/25, Train Loss: 0.0982, Val Loss: 0.0503, Val Acc: 0.9840
Epoch 11/25, Train Loss: 0.1018, Val Loss: 0.0479, Val Acc: 0.9760
Epoch 12/25, Train Loss: 0.0999, Val Loss: 0.0511, Val Acc: 0.9760
Epoch 13/25, Train Loss: 0.0740, Val Loss: 0.0422, Val Acc: 0.9920
Epoch 14/25, Train Loss: 0.0786, Val Loss: 0.0326, Val Acc: 0.9920
Epoch 15/25, Train Loss: 0.0920, Val Loss: 0.0491, Val Acc: 0.9840
Epoc

In [69]:
import torch
from torchvision import datasets, transforms

# Define the transform
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
batch_size = 32
test_dataset = datasets.ImageFolder('/kaggle/input/cv-assignment/group_3/test', test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy: {:.2f}%'.format(accuracy))



Accuracy: 100.00%


In [70]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# define the device to be used for inference
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# load the image and apply the required transformations
image_path = '/kaggle/input/cv-assignment/group_3/train/baseball/001.jpg'
image = Image.open(image_path).convert('RGB')
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])
image_tensor = transform(image).unsqueeze(0)

# load the custom trained model
import torch
import torch.nn as nn
import os
from torchvision.models import vgg16

# define the architecture of the model
model = vgg16(pretrained=False)
model.classifier[6] = nn.Linear(4096, 25)  # modify the last layer for binary classification

# load the saved state dict into the model
state_dict = torch.load('/kaggle/working/epoch25.pt')
model.load_state_dict(state_dict)

# move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# perform inference on the image
with torch.no_grad():
    image_tensor = image_tensor.to(device)
    outputs = model(image_tensor)
    _, predicted = torch.max(outputs.data, 1)

# print the predicted class label
print('Predicted class id:', predicted.item())

test_dir = '/kaggle/input/cv-assignment/group_3/test/'

# get the list of class names from the folder names in the test directory
class_names = sorted(os.listdir(test_dir))
predicted_name = class_names[predicted.item()]
print('Predicted class name:', predicted_name)

Predicted class id: 1
Predicted class name: baseball


In [71]:
#L2 regularization

import torch.nn as nn
import torch.optim as optim

model = vgg16()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)



In [72]:
#L1

optimizer = optim.Adam(model.classifier[-1].parameters(), lr=0.001, weight_decay=0.0001)
